<a id='TOP'></a> *virtual environment: FERpython*
# PC registration using both spatial and color
When it comes to point cloud registration, leveraging both spatial (geometric) and intensity or color information can provide more robust and accurate alignments, especially when dealing with datasets that have distinct intensity or color patterns.

Here are some methods that consider both spatial and color/intensity attributes for point cloud registration:

1. [**Colored ICP(Iterative Closest Point)**](#Colored_ICP)


2. [**Generalized ICP (GICP) with color**](#GICP)


3. [**Joint Color and Depth (JCD) Descriptors**](#JCD_descriptors)

    3.1 [Compute JCD function](#JCD01)
    
    3.2 [Feature Extraction](#JCD02)
    
    3.3 [Feature Matching](#JCD03)
    
    3.4 [Computing the transformation](#JCD04)
    
    
4. [**Coherent Point Drift with Color (Colored CPD)**](#Coherent_Point_Drift)


5. [**RANSAC with Color-based Descriptors**](#RANSAC_Color_based)

6. [**4-Points Congruent Sets (4PCS) and its Color-based Variants**](#4PCS)

7. [**Deep Learning Approaches**](#DL)
    1. [PointNetLK](#PointLK)
    2. [2](#2)
    3. [3](#3)
8. [**GA-Based Optimization for Color Point Cloud Registration**](#GA-based)

In [2]:
%run Mylib_import.ipynb

#Import pointcloud
source_point_cloud = o3d.io.read_point_cloud("ply_test/frag_115.ply")
target_point_cloud = o3d.io.read_point_cloud("ply_test/frag_116.ply")
# source_point_cloud = o3d.io.read_point_cloud("ply_test/00520-depth.ply")#
# target_point_cloud = o3d.io.read_point_cloud("ply_test/00540-depth.ply")#
# source_point_cloud = o3d.io.read_point_cloud("ply_test/bun000.ply")#
# target_point_cloud = o3d.io.read_point_cloud("ply_test/bun045.ply")#
# source_point_cloud.paint_uniform_color([1, 0, 0])  # [R, G, B] where each value is from 0 to 1
# target_point_cloud.paint_uniform_color([0, 0, 1])


In [3]:
import numpy as np
import open3d as o3d

def compute_point_cloud_resolution(pcd):
    """
    Compute the resolution of a point cloud, defined as the average
    distance between each point and its nearest neighbor.
    """
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    return avg_dist

def set_distance_threshold(source_pcd, target_pcd, multiplier=1.0):
    """
    Set distance threshold relative to the point clouds' resolution.
    """
    source_res = compute_point_cloud_resolution(source_pcd)
    target_res = compute_point_cloud_resolution(target_pcd)
    avg_res = (source_res + target_res) / 2.0
    return avg_res * multiplier


# Compute the distance threshold
dist_resolution = set_distance_threshold(source_point_cloud, target_point_cloud, multiplier=1.0)
dist_resolution

0.0037137852441603107

In [74]:
import numpy as np
import open3d as o3d

def compute_point_cloud_size(pcd):
    """
    Compute the size of the point cloud along each dimension.
    """
    points = np.asarray(pcd.points)
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)
    size = max_coords - min_coords
    return size

def compute_point_cloud_radius(pcd):
    """
    Compute the radius of the point cloud.
    """
    points = np.asarray(pcd.points)
    centroid = np.mean(points, axis=0)
    distances = np.linalg.norm(points - centroid, axis=1)
    radius = np.max(distances)
    return radius

def set_distance_threshold_based_on_size_or_radius(source_pcd, target_pcd, size_multiplier=0.01, radius_multiplier=0.01):
    """
    Set distance threshold based on point cloud size or radius.
    """
    source_size = compute_point_cloud_size(source_pcd)
    target_size = compute_point_cloud_size(target_pcd)
    avg_size = (np.mean(source_size) + np.mean(target_size)) / 2.0

    source_radius = compute_point_cloud_radius(source_pcd)
    target_radius = compute_point_cloud_radius(target_pcd)
    avg_radius = (source_radius + target_radius) / 2.0

    # You can choose to set threshold based on size or radius
    distance_threshold_size_based = avg_size * size_multiplier
    distance_threshold_radius_based = avg_radius * radius_multiplier

    # Return the chosen threshold
    return distance_threshold_size_based  # or distance_threshold_radius_based


# Compute the distance threshold
dist_radius = set_distance_threshold_based_on_size_or_radius(source_point_cloud, target_point_cloud)
dist_radius

0.0014412386666666666

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='Colored_ICP'></a>
# 1. Colored ICP (Iterative Closest Point)

- A natural extension of the standard ICP algorithm.
- The registration process minimizes a combined error that includes both geometric distance and color (or intensity) difference between matched points.

Implementing Colored ICP involves modifying the traditional ICP to incorporate color information during the registration process. Here's a basic implementation of the Colored ICP in Python using **numpy** and **open3d**.

here's a simplified version of the Colored ICP:

In [75]:
%run Mylib_import.ipynb
import numpy as np
import open3d as o3d

def colored_icp(source, target, max_iterations=30, distance_threshold=0.02, color_weight=0.3):
    """
    Colored ICP registration.
    """
    print('max_iterations:', max_iterations)
    print('distance_threshold:', distance_threshold)
    print('color_weight:', color_weight)
    transformation = np.eye(4)
    prev_error = float('inf')

    if not source.has_colors() or not target.has_colors():
        raise ValueError("Source or target point cloud doesn't have color information.")

    for iteration in range(max_iterations):
        distances = source.compute_point_cloud_distance(target)
        correspondences = np.asarray(distances).argsort()
        
        correspondence_distances = np.asarray(distances)[correspondences]
        valid_ids = np.where(correspondence_distances < distance_threshold)[0]
        
        if valid_ids.size == 0:
            print(f"No valid correspondences found in iteration {iteration}.")
            break
        
        # Filtering both correspondences and valid_ids
        correspondences = correspondences[valid_ids]
        valid_ids = valid_ids[correspondences < len(target.points)]
        correspondences = correspondences[correspondences < len(target.points)]
        
        source_points = np.asarray(source.points)[valid_ids, :]
        target_points = np.asarray(target.points)[correspondences, :]
        
        source_colors = np.asarray(source.colors)[valid_ids, :]
        target_colors = np.asarray(target.colors)[correspondences, :]
        
        color_diff = source_colors - target_colors
        color_errors = np.linalg.norm(color_diff, axis=1)
        geometric_errors = correspondence_distances[valid_ids]
        
        combined_errors = (1.0 - color_weight) * geometric_errors + color_weight * color_errors
        
        if np.mean(combined_errors) >= prev_error:
            break

        prev_error = np.mean(combined_errors)
        
        est_transformation = o3d.pipelines.registration.registration_icp(
            source, target, distance_threshold, transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(
                with_scaling=False)).transformation
        
        source.transform(est_transformation)
        transformation = np.dot(est_transformation, transformation)

    return transformation



# Test the function
if __name__ == '__main__':
    source = copy.deepcopy(source_point_cloud)
    target = copy.deepcopy(target_point_cloud)
    transformed_source = copy.deepcopy(source_point_cloud)
    transformation = colored_icp(source, target, 100, 0.01, 0.3)
#     transformation = colored_icp(source, target, 100, dist_resolution, 0.5)
#     transformation = colored_icp(source, target, 100, dist_radius, 0.5)
    print(transformation)
    transformed_source.transform(transformation)
    o3d.visualization.draw_geometries([source_point_cloud + target_point_cloud],width=800, height=600)
    o3d.visualization.draw_geometries([transformed_source + target_point_cloud],width=800, height=600)


max_iterations: 100
distance_threshold: 0.01
color_weight: 0.3
[[ 8.57811630e-01  1.06479853e-02 -5.13853897e-01  3.94841158e-02]
 [-5.11560293e-03  9.99912733e-01  1.21801789e-02 -4.00019653e-04]
 [ 5.13938749e-01 -7.81962658e-03  8.57791243e-01  4.00921292e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


Replace **"path_to_source.pcd"** and **"path_to_target.pcd"** with the paths to your source and target point cloud files.

This is a simple version of the Colored ICP. For more efficiency and features (like handling large point clouds or further optimizing the process), advanced data structures, multi-threading, or even GPU acceleration could be utilized. The parameter color_weight can be adjusted to give more or less weight to the color term during registration. Adjusting the parameters and thresholds may also help improve registration for specific datasets.

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='GICP'></a>
# 2. Generalized ICP (GICP) with color
Combining Generalized ICP (GICP) with color, it isn't natively supported by the standard libraries such as Open3D or PCL (Point Cloud Library). But we can craft a combined approach similar to what we discussed for Colored ICP.

Here's a conceptual strategy for combining GICP with color:

- **Initial Alignment with Color:** Use color-based features for a coarse registration to provide an initial alignment. You can use feature descriptors like FPFH, SHOT, or other color-aware descriptors.


- **Refinement with GICP:** With the initial alignment as a starting point, employ GICP to refine the registration, taking advantage of both the geometry and the color.

Let's outline a Python function to achieve this, using Open3D:


[(BACK TO TOP)](#TOP)

In [76]:
%run Mylib_import.ipynb
import open3d as o3d
import numpy as np

def gicp_with_color(source, target, distance_threshold=0.02, max_iter=30,
                    radius_normal = 0.02, radius_feature = 0.02):
    """
    Register source to target using initial color alignment followed by GICP refinement.
    :param source: Source point cloud
    :param target: Target point cloud
    :param distance_threshold: Maximum correspondence distance for GICP
    :return: Transformation matrix (source to target)
    """
    print('distance_threshold:', distance_threshold)
    print('max_iter:', max_iter)
    print('radius_normals:', radius_normal)
    print('radius_feature:', radius_feature)
    
    print("Checking source point cloud...")
    source = check_and_compute_normals(source,distance_threshold)
    print("\nChecking target point cloud...")
    target = check_and_compute_normals(target,distance_threshold)   
    
    # Initial Alignment using RANSAC and color-based features like FPFH
    source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        source, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_iter))
    target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        target, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_iter))
    result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, True, distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        4,  # Number of points to estimate a transform from
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
    )

    # Refinement using GICP
    source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,
                                                                              max_nn=max_iter))
    target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,
                                                                              max_nn=max_iter))
    result_gicp = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )

    return result_gicp.transformation

def check_and_compute_normals(pcd, radius_normal=0.02, max_iter=30):
    """Check if a point cloud has normals, and compute them if not."""
    print('radius_normals:', radius_normal)
    if not pcd.has_normals():
        print("Point cloud doesn't have normals. Computing...")
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_iter))
        print("Normals computed!")
    else:
        print("Point cloud already has normals.")
    return pcd

# Test the function
if __name__ == '__main__':
    source = copy.deepcopy(source_point_cloud)
    target = copy.deepcopy(target_point_cloud)
#     source = o3d.io.read_point_cloud("ply_test/frag_115.ply")
#     target = o3d.io.read_point_cloud("ply_test/frag_116.ply")
    transformed_source = copy.deepcopy(source)
    transformation = gicp_with_color(source, target, distance_threshold=0.008, max_iter=30, 
                                     radius_normal = 0.01, radius_feature = 0.1) #0.008
    print(transformation)
    transformed_source.transform(transformation)
#     o3d.visualization.draw_geometries([source + target],width=800, height=600)
#     o3d.visualization.draw_geometries([transformed_source + target],width=800, height=600)

distance_threshold: 0.008
max_iter: 30
radius_normals: 0.01
radius_feature: 0.1
Checking source point cloud...
radius_normals: 0.008
Point cloud doesn't have normals. Computing...
Normals computed!

Checking target point cloud...
radius_normals: 0.008
Point cloud doesn't have normals. Computing...
Normals computed!
[[ 8.25771998e-01  5.20498685e-03 -5.63980067e-01  3.64528605e-02]
 [-1.33143501e-02  9.99858649e-01 -1.02669699e-02 -1.97632987e-04]
 [ 5.63846908e-01  1.59872043e-02  8.25724575e-01  3.79134478e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [77]:
o3d.visualization.draw_geometries([source_point_cloud + target_point_cloud],width=800, height=600)
o3d.visualization.draw_geometries([transformed_source + target_point_cloud],width=800, height=600)

This approach uses FPFH features to capture initial color-based alignments and then GICP refines it by considering geometry. This is a hybrid method and might not fully exploit the synergy of color and geometry as a dedicated algorithm would, but it can give satisfactory results in many scenarios. Adjusting parameters and thresholds might be necessary based on the nature and quality of your datasets.

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='JCD_descriptors'></a>
# 3. Joint Color and Depth (JCD) Descriptors

-This method uses local surface properties and RGB color values to compute descriptors for each point.

-These descriptors can then be matched across point clouds to find correspondences for registration.


[(BACK TO TOP)](#TOP)

<a id='JCD01'></a>
# 3.1 Compute JCD function

In [78]:
%run Mylib_import.ipynb
import open3d as o3d
import numpy as np

def compute_JCD(point_cloud, point, radius):
    """
    Compute the JCD descriptor for a given point in a point cloud.
    :param point_cloud: The input point cloud with color and depth.
    :param point: The reference point for which the descriptor is computed.
    :param radius: The radius to consider for neighboring points.
    :return: JCD descriptor
    """

    neighbors = find_neighbors_within_radius(point_cloud, point, radius)

    # Compute depth-based features
    depth_features = compute_depth_features(point_cloud, point, neighbors)
    
    # Compute color-based features
    query_color_idx = np.where(np.linalg.norm(np.array(point_cloud.points) - point, axis=1) < 1e-4)[0]
    if len(query_color_idx) == 0:
        return
        #raise ValueError("The query point is not in the point cloud.")
    query_color = np.asarray(point_cloud.colors)[query_color_idx[0]]
    color_features = compute_color_features(point_cloud, query_color, neighbors)

    # Concatenate depth and color features to form JCD
    jcd_descriptor = concatenate(depth_features, color_features)

    return jcd_descriptor

def find_neighbors_within_radius(point_cloud, query_point, radius):
    """
    Find neighbors of a query point within a given radius in a point cloud.
    
    :param point_cloud: The input point cloud.
    :param query_point: The point for which neighbors are queried.
    :param radius: Search radius.
    
    :return: Indices of the neighboring points within the radius.
    """
    
    # Construct a KDTree
    kdtree = o3d.geometry.KDTreeFlann(point_cloud)
    
    # Query the KDTree to get points within the radius
    _, indices, _ = kdtree.search_radius_vector_3d(query_point, radius)
    
    return indices

def compute_depth_features(point_cloud, query_point, neighbors_indices):
    """
    Compute depth-based (geometric) features for a point using its neighbors.
    
    :param point_cloud: The input point cloud with normals.
    :param query_point: The point for which features are computed.
    :param neighbors_indices: Indices of neighboring points.
    
    :return: Depth-based features [avg_distance, std_distance, avg_normal_variation]
    """
    
    # Extract the coordinates of neighbors
    neighbors = np.asarray(point_cloud.points)[neighbors_indices]
    
    # Compute distances from query point to each neighbor
    distances = np.linalg.norm(neighbors - query_point, axis=1)
    
    # Compute average and standard deviation of distances
    avg_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Compute normal variation
    kdtree = o3d.geometry.KDTreeFlann(point_cloud)
    _, matching_indices, _ = kdtree.search_knn_vector_3d(query_point, 1)

    if len(matching_indices) == 0:
        raise ValueError("The query point is not in the point cloud.")
    query_normal = np.asarray(point_cloud.normals)[matching_indices[0]]
    neighbors_normals = np.asarray(point_cloud.normals)[neighbors_indices]
    normal_differences = np.arccos(np.clip(np.dot(neighbors_normals, query_normal), -1.0, 1.0))
    avg_normal_variation = np.mean(normal_differences)
    
    return [avg_distance, std_distance, avg_normal_variation]


def compute_color_features(point_cloud, query_color, neighbors_indices):
    """
    Compute color-based features for a point using its neighbors.
    
    :param point_cloud: The input point cloud with colors.
    :param query_color: Color of the query point.
    :param neighbors_indices: Indices of neighboring points.
    
    :return: Color-based features [avg_color_diff, std_color_diff, color_histogram]
    """
    
    # Extract the colors of neighbors
    neighbors_colors = np.asarray(point_cloud.colors)[neighbors_indices]
    
    # Compute color differences
    color_differences = np.linalg.norm(neighbors_colors - query_color, axis=1)
    
    # Average and standard deviation of color differences
    avg_color_diff = np.mean(color_differences)
    std_color_diff = np.std(color_differences)
    
    # Compute color histogram (for simplicity, using the R channel and 10 bins here)
    color_histogram, _ = np.histogram(neighbors_colors[:, 0], bins=10, range=(0, 1))
    
    return [avg_color_diff, std_color_diff, color_histogram]

def concatenate(depth_features, color_features):
    """
    Concatenate depth and color features to form the JCD descriptor.
    
    :param depth_features: List of depth-based features.
    :param color_features: List of color-based features.
    
    :return: Combined JCD descriptor as a numpy array.
    """
    return np.concatenate((depth_features, np.array(color_features[:-1]), color_features[-1]))

Using the Joint Color and Depth (JCD) descriptors for point cloud registration involves a sequence of steps, including feature extraction, feature matching, and transformation estimation. Here's a high-level approach to use the JCD descriptors for point cloud registration:

<a id='JCD02'></a>
# **3.2 Feature Extraction:**

Extract JCD descriptors for keypoints in both the source and target point clouds. Keypoints can be selected using various methods like random sampling, uniform grid sampling, or using algorithms like SIFT for point clouds.

[(BACK TO TOP)](#TOP)

In [4]:
source = copy.deepcopy(source_point_cloud)
target = copy.deepcopy(target_point_cloud)
# source_point_cloud = o3d.io.read_point_cloud("ply_test/frag_115.ply") 
# target_point_cloud = o3d.io.read_point_cloud("ply_test/frag_116.ply") 

**SIFT Feature Extraction:** using SIFT to extract keypoints from the pseudo-images

In [5]:
import cv2
def point_cloud_to_color_and_point_map(pcd, img_dim=200): #200
    # Convert point cloud data to numpy array
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) *255  # Scaling to 0-255 range
    
    # Normalize x, y to image dimensions
    normalized_points = (((points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))) * (img_dim - 1)).astype(int)
    img_coords = normalized_points[:, :2].astype(int)
    
    # Create color map and count map for averaging
    color_map = np.zeros((img_dim, img_dim, 3))
    count_map = np.zeros((img_dim, img_dim, 3))

    # Create point_map to store 3D coordinates
    point_map = np.zeros((img_dim, img_dim, 3))
    
    for img_coord, color, point in zip(img_coords, colors, points):
        color_map[img_coord[1], img_coord[0]] += color  # accumulate color values
        count_map[img_coord[1], img_coord[0]] += 1  # count the number of points per bin

        # For point map, just store the 3D point for now without averaging
        # This could be changed based on specific requirements
        point_map[img_coord[1], img_coord[0]] = point

    # Average the accumulated color values
    avg_mask = count_map > 0
    color_map[avg_mask] /= count_map[avg_mask]
    
    return color_map.astype(np.uint8), point_map

def extract_sift_features_tweaked(img):
    gray = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2GRAY)
    # Reduce contrastThreshold and edgeThreshold to capture more features (possibly weaker ones)
    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.08, edgeThreshold=0.2)#(contrastThreshold=0.02, edgeThreshold=5)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

def draw_and_show_keypoints(img, keypoints, title='Keypoints'):
    img_with_keypoints = cv2.drawKeypoints(img, keypoints, outImage=np.array([]), color=(0, 255, 0))
    cv2.imshow(title, img_with_keypoints)

# Generate pseudo-images (color maps) for both point clouds:    
src_pseudo_color_map, src_pseudo_point_map = point_cloud_to_color_and_point_map(source)
tar_pseudo_color_map, tar_pseudo_point_map  = point_cloud_to_color_and_point_map(target)

# Extracting SIFT keypoints and descriptors: from the pseudo-images.   
src_pseudo_keypoints, src_pseudo_descriptors = extract_sift_features_tweaked(src_pseudo_color_map)
tar_pseudo_keypoints, tar_pseudo_descriptors = extract_sift_features_tweaked(tar_pseudo_color_map)

draw_and_show_keypoints(src_pseudo_color_map, src_pseudo_keypoints, 'Source pseudo-image Keypoints')
draw_and_show_keypoints(tar_pseudo_color_map, tar_pseudo_keypoints, 'Target pseudo-image Keypoints')
cv2.waitKey(0)
cv2.destroyAllWindows()


<a id='JCD03'></a>
# 3.3 Feature Matching: 
Match the descriptors from the source and target point clouds.

[(BACK TO TOP)](#TOP)

Compute JCD descriptors and get 3D matching 

In [6]:
def compute_jcd_descriptor(sift_descriptors, point_map, keypoints): #2D
    """
    Compute JCD descriptor by appending normalized depth information (z-coordinate) to SIFT descriptors.
    """
    jcd_descriptors = []
    
    # Normalize the depth values
    depth_values = [point_map[int(kp.pt[1]), int(kp.pt[0]), 2] for kp in keypoints]
    max_depth = max(depth_values)
    min_depth = min(depth_values)
    
    for keypoint, descriptor in zip(keypoints, sift_descriptors):
        # Extract (x, y) from the keypoint
        x, y = int(keypoint.pt[0]), int(keypoint.pt[1])
        
        # Extract and normalize depth (z-coordinate) from the point_map
        depth = (point_map[y, x, 2] - min_depth) / (max_depth - min_depth)
        
        # Convert descriptor to float type and append normalized depth to the descriptor
        jcd_descriptor = np.concatenate([descriptor.astype(float), np.array([depth])])
        
        jcd_descriptors.append(jcd_descriptor)
    
    return np.array(jcd_descriptors)


# Computing JCD descriptors for source and target:
src_jcd_descriptors = compute_jcd_descriptor(src_pseudo_descriptors, src_pseudo_point_map, src_pseudo_keypoints)
tar_jcd_descriptors = compute_jcd_descriptor(tar_pseudo_descriptors, tar_pseudo_point_map, tar_pseudo_keypoints)

src_jcd_descriptors = src_jcd_descriptors.astype(np.float32)
tar_jcd_descriptors = tar_jcd_descriptors.astype(np.float32)

# # Matching JCD descriptors using a Brute Force Matcher:      
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.match(src_jcd_descriptors, tar_jcd_descriptors)

#----------------------------
# Visualize matches in 2D
# Draw top N matches (e.g., N = 50)
img_matches = cv2.drawMatches(src_pseudo_color_map, src_pseudo_keypoints,
                              tar_pseudo_color_map, tar_pseudo_keypoints, matches[:50], None)
cv2.imshow("Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

#-------------------------------------------------------------
def get_3d_matches(matches, src_keypoints, tar_keypoints, src_point_map, tar_point_map):
    src_3d_matches = []
    tar_3d_matches = []
    
    for match in matches:
        # Extracting the 2D keypoint from source and target
        src_kp = src_keypoints[match.queryIdx]
        tar_kp = tar_keypoints[match.trainIdx]
        
        # Using the 2D keypoints to look up the 3D coordinates in the point maps
        src_x, src_y = int(src_kp.pt[0]), int(src_kp.pt[1])
        tar_x, tar_y = int(tar_kp.pt[0]), int(tar_kp.pt[1])
        
        src_3d_point = src_point_map[src_y, src_x]
        tar_3d_point = tar_point_map[tar_y, tar_x]
        
        src_3d_matches.append(src_3d_point)
        tar_3d_matches.append(tar_3d_point)
        
    return src_3d_matches, tar_3d_matches

# Get 3D matches from 2D matches(from jcd descriptors), pseudo-keypoints, and point maps
src_3d_matches, tar_3d_matches = get_3d_matches(matches, src_pseudo_keypoints,
                                                tar_pseudo_keypoints, src_pseudo_point_map,
                                                tar_pseudo_point_map)

#----------------------------------------------------
#Visualize matches in 3D
# Assign different colors for source and target point clouds
source = copy.deepcopy(source_point_cloud)
target = copy.deepcopy(target_point_cloud)

# Convert matches to numpy arrays
src_3d_matches_np = np.array(src_3d_matches)
tar_3d_matches_np = np.array(tar_3d_matches)

# Prepare lines and colors for the LineSet
lines = [[i, i + len(src_3d_matches_np)] for i in range(len(src_3d_matches_np))]
colors = [[0, 1, 0] for i in range(len(src_3d_matches_np))]  # Green for matches

# Create a LineSet from matched pairs
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.vstack([src_3d_matches_np, tar_3d_matches_np]))
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Visualize
o3d.visualization.draw_geometries([source_point_cloud, target_point_cloud, line_set],width=800, height=600)

<a id='JCD04'></a>
# 3.4 Computing the transformation:
between the two point clouds based on these matches.


[(BACK TO TOP)](#TOP)

In [8]:
#Using ICP
import open3d as o3d
import numpy as np

# Convert matched points to PointCloud
src_match_pcd = o3d.geometry.PointCloud()
src_match_pcd.points = o3d.utility.Vector3dVector(src_3d_matches)

tar_match_pcd = o3d.geometry.PointCloud()
tar_match_pcd.points = o3d.utility.Vector3dVector(tar_3d_matches)

# Estimate the transformation
reg_p2p = o3d.registration.registration_icp(
    src_match_pcd, tar_match_pcd, 0.09, np.eye(4), #0.09 (flag)
    o3d.registration.TransformationEstimationPointToPoint())

# Get the transformation matrix
transformation_icp = reg_p2p.transformation

print(transformation_icp)

trans_src = copy.deepcopy(source_point_cloud)
trans_src.transform(transformation_icp)
o3d.visualization.draw_geometries([source_point_cloud, target_point_cloud],width=800, height=600)
o3d.visualization.draw_geometries([trans_src, target_point_cloud],width=800, height=600)

[[ 9.99969665e-01  1.09896054e-04 -7.78828824e-03  3.00268927e-02]
 [ 4.26419001e-04  9.97628546e-01  6.88266137e-02  8.96512087e-03]
 [ 7.77738244e-03 -6.88278469e-02  9.97598236e-01  1.95419752e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [13]:
#Using RANSAC (difficult settings to get good result)
import open3d as o3d
import numpy as np

# Convert matched points to PointCloud
src_match_pcd = o3d.geometry.PointCloud()
src_match_pcd.points = o3d.utility.Vector3dVector(src_3d_matches)

tar_match_pcd = o3d.geometry.PointCloud()
tar_match_pcd.points = o3d.utility.Vector3dVector(tar_3d_matches)

# Convert matches to correspondence set
corres = o3d.utility.Vector2iVector(np.column_stack([range(len(src_3d_matches)), range(len(tar_3d_matches))]))

# Use RANSAC for registration
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

result_rs = o3d.registration.registration_ransac_based_on_correspondence(
    src_match_pcd, tar_match_pcd, corres, 
    max_correspondence_distance=0.09, 
    estimation_method=o3d.registration.TransformationEstimationPointToPoint(False), 
    ransac_n=4,
    criteria=o3d.registration.RANSACConvergenceCriteria(4000000, 500)
)

# Get the transformation matrix
transformation_rs = result_rs.transformation
print(transformation_rs)

trans_src = copy.deepcopy(source_point_cloud)
trans_src.transform(transformation_rs)
o3d.visualization.draw_geometries([source_point_cloud, target_point_cloud],width=800, height=600)
o3d.visualization.draw_geometries([trans_src, target_point_cloud],width=800, height=600)

[[ 0.99988191 -0.01159777 -0.01008285  0.060381  ]
 [ 0.01215174  0.99831517  0.05673767 -0.02036119]
 [ 0.00940783 -0.0568535   0.99833821  0.16337022]
 [ 0.          0.          0.          1.        ]]


**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='Coherent_Point_Drift'></a>
# 4. Coherent Point Drift with Color (Colored CPD)

What the code does:
1. The code appears to implement a variant of the Coherent Point Drift (CPD) algorithm with color as an additional feature for alignment.

2. The class **'ColoredCPD'** contains the main functions to achieve this. The point clouds are downsampled, and then registration is performed through a series of iterations.

3. The method **'compute_probabilities'** is responsible for computing the probabilities for correspondence between points in the source and target point clouds.

4. **'update_transformation'** method calculates and updates the transformation that aligns the source point cloud to the target point cloud.

5. Finally, the **'register'** method runs the algorithm for a defined number of iterations.

**'weight'** controls the balance between a uniform distribution and the data term for point matching, 

while **color_weight** controls how important color matching is in the algorithm.

Both are important depending on what you want to emphasize in your point cloud registration task.


[(BACK TO TOP)](#TOP)

In [88]:
import numpy as np
import open3d as o3d
import copy

class ColoredCPD:
    
    def __init__(self, target, source, voxel_size=0.02, weight=0.2, color_weight=1.0,
                 sigma=0.1, iterations=10):
        # Downsample point clouds
        self.target = target.voxel_down_sample(voxel_size) 
        self.source = source.voxel_down_sample(voxel_size)
        self.weight = weight
        self.color_weight = color_weight
        self.transformation_matrices = []
        self.sigma = sigma # standard deviation for Gaussian in probability computation
        self.iterations = iterations
        
    def compute_probabilities(self):
        # Use KDTree for efficient nearest neighbor search
        target_tree = o3d.geometry.KDTreeFlann(self.target)
        
        # Placeholder for probabilities
        P = np.zeros((len(self.source.points), len(self.target.points)))

        for i, point in enumerate(self.source.points):
            # Find k nearest points; for simplicity, we consider k=1 here
            _, idx, _ = target_tree.search_knn_vector_3d(point, 1)
            nearest_point = np.asarray(self.target.points)[idx, :]
            nearest_color = np.asarray(self.target.colors)[idx, :]
            
            # Calculate color difference
            color_diff = np.linalg.norm(np.asarray(self.source.colors)[i] - nearest_color)
            
            # Assume Gaussian distribution for color difference
            prob = np.exp(-self.color_weight * color_diff ** 2 / (2 * self.sigma ** 2))

            # Adjust the probability based on the weight parameter
            P[i, idx] = (1 - self.weight) * len(self.source.points) * prob / (self.weight + (1 - self.weight) * prob)

        return P

    def update_transformation(self, P):
        # Calculate centroids of source and target points based on probabilities
        p1 = np.sum(P, axis=1)
        p2 = np.sum(P, axis=0)

        muX = np.dot(np.asarray(self.source.points).T, p1) / np.sum(p1)
        muY = np.dot(np.asarray(self.target.points).T, p2) / np.sum(p2)

        X_hat = np.asarray(self.source.points) - muX
        Y_hat = np.asarray(self.target.points) - muY

        W = np.dot(X_hat.T, P).dot(Y_hat)
        U, _, Vt = np.linalg.svd(W)
        R = np.dot(Vt.T, U.T)
        t = muY - np.dot(R, muX)

        # Update source points
        transformed_points = np.dot(R, np.asarray(self.source.points).T).T + t
        self.source.points = o3d.utility.Vector3dVector(transformed_points)

        # Create and store the 4x4 transformation matrix
        self.transformation_matrix = np.eye(4)
        self.transformation_matrix[0:3, 0:3] = R
        self.transformation_matrix[0:3, 3] = t
        self.transformation_matrices.append(self.transformation_matrix)

    def register(self): #10
        for _ in range(self.iterations):
            P = self.compute_probabilities()
            self.update_transformation(P)
        
        return self.source
    
    def get_final_transformation(self):
        final_transform = np.eye(4)
        for T in self.transformation_matrices:
            final_transform = np.dot(final_transform, T)
        return final_transform


# Load point clouds
source = copy.deepcopy(source_point_cloud)
target = copy.deepcopy(target_point_cloud)
# source = o3d.io.read_point_cloud("ply_test/frag_115.ply")
# target = o3d.io.read_point_cloud("ply_test/frag_116.ply")
# Deep copy the source point cloud after registration
transformed_source = copy.deepcopy(source)

# Initialize CPD and register
# cpd = ColoredCPD(target, source, voxel_size=0.9, weight=0.2, color_weight=0.5,sigma=1.0, iterations=30)                
cpd = ColoredCPD(target, source, voxel_size=0.1, weight=0.01, color_weight=10,sigma=0.1, iterations=30)                

cpd.register()

# Visualize the original point clouds
o3d.visualization.draw_geometries([source+target], width=800, height=600)

# Apply the transformation to the deep-copied point cloud
final_transformation = cpd.get_final_transformation()
transformed_source.transform(final_transformation)

# Visualize the transformed source with the target
o3d.visualization.draw_geometries([transformed_source+target], width=800, height=600)
final_transformation

C:\Users\msari\.conda\envs\pointnetlk_env\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pointnetlk_env\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide


LinAlgError: SVD did not converge

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='RANSAC_Color_based'></a>
# 5. RANSAC with Color-based Descriptors

The goal of the code is to align two point clouds based on the similarity of their colors using the RANSAC algorithm. 

1. **Imports and Point Cloud Loading**:
You imported necessary libraries and loaded the point clouds.

2. **Downsampling**:
You downsampled the source and target point clouds to make further computations faster.

3. **Color-based Correspondence Search**:
- You converted the colors of the target point cloud into a separate point cloud (target_colors_pcd) to facilitate KDTree-based searches.
- Using a KDTree, you found the closest color matches for every color in the source point cloud within the target color point cloud.

4. **Filtering Out Invalid Correspondences**:
You created a list of valid correspondences by checking if any of the RGB channels in both source and target points were non-zero (indicating the presence of a valid color).

5. **RANSAC Registration**:
Using the valid correspondences, you performed a RANSAC-based registration to find the optimal transformation to align the source point cloud to the target point cloud.

6. **Visualization**:
Lastly, you applied the computed transformation to the original (not downsampled) source point cloud and visualized it alongside the target point cloud.



[(BACK TO TOP)](#TOP)

In [90]:
import open3d as o3d
import numpy as np
import copy
    

# Load point clouds
# source = o3d.io.read_point_cloud("ply_test/frag_115.ply")
# target = o3d.io.read_point_cloud("ply_test/frag_116.ply")

voxel_size = 0.5  # adjust based on your needs
source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

# Convert RGB colors to a PointCloud object
target_colors_pcd = o3d.geometry.PointCloud()
target_colors_pcd.points = o3d.utility.Vector3dVector(np.asarray(target_down.colors))

source_colors = np.asarray(source_down.colors)

# Using KDTree to find correspondences based on color similarity
tree = o3d.geometry.KDTreeFlann(target_colors_pcd)
correspondences = [tree.search_knn_vector_3d(color, 1)[1][0] for color in source_colors]

# Convert Vector3dVector to numpy array for slicing
target_colors_array = np.asarray(target_down.colors)

correspondence_set = [(i, correspondences[i]) for i in range(len(source_colors)) 
                      if source_colors[i, :3].any() 
                      and target_colors_array[correspondences[i], :3].any()]

# Convert correspondence set into the expected format for RANSAC
correspondence_set_vector = o3d.utility.Vector2iVector(correspondence_set)

distance_threshold = 0.02
ransac_result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
    source_down, 
    target_down, 
    correspondence_set_vector, 
    distance_threshold, 
    ransac_n=4, 
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 50)
)

# Draw results
transformed_source = copy.deepcopy(source)  # Create a copy so original source isn't modified
transformed_source.transform(ransac_result.transformation)
    
o3d.visualization.draw_geometries([transformed_source, target], width=800, height=600)

In [42]:
ransac_result.transformation
# o3d.visualization.draw_geometries([source, target], width=800, height=600)

array([[ 0.55784107,  0.50769383, -0.65655184,  0.3930332 ],
       [ 0.45752152, -0.84813298, -0.26710394,  3.41401941],
       [-0.69245029, -0.15138505, -0.70540426,  3.40224544],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='4PCS'></a>
# 6. 4-Points Congruent Sets (4PCS) and its Color-based Variants

please note the following:

1. The function *'color_difference'* computes the color difference in RGB space.
2. The *'is_congruent'* function now considers both geometry (point positions) and color.
3. *'color_threshold'* is an arbitrary value I've set as 30 for this example; you might want to adjust it based on your needs.
4. This code uses colors in the congruency check but doesn't use colors directly in the RANSAC registration. Integrating color directly into the RANSAC procedure would require a more sophisticated method.

[(BACK TO TOP)](#TOP)

In [92]:
import open3d as o3d
import numpy as np
import itertools
import copy

def compute_bases(point_set, delta):
    return list(itertools.combinations(range(len(point_set.points)), 4))

def color_difference(color1, color2):
    return np.linalg.norm(np.array(color1) - np.array(color2))

def is_congruent(base1_indices, base2_indices, points1, points2, colors1, colors2, delta, color_threshold):
    for i in range(4):
        for j in range(4):
            if i != j:
                dist1 = np.linalg.norm(np.array(points1[base1_indices[i]]) - np.array(points1[base1_indices[j]]))
                dist2 = np.linalg.norm(np.array(points2[base2_indices[i]]) - np.array(points2[base2_indices[j]]))
                color_diff = color_difference(colors1[base1_indices[i]], colors2[base2_indices[j]])
                
                if abs(dist1 - dist2) > delta or color_diff > color_threshold:
                    return False
    return True

def estimate_transformation(source, target, threshold=1.0):
    distance_threshold = threshold
    result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
        source, target, o3d.utility.Vector2iVector(np.array(list(itertools.product(range(len(source.points)), range(len(target.points)))))), 
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        4,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )
    return result.transformation

def random_base(point_set, n=4):
    """Sample n random points from the point cloud."""
    indices = np.random.choice(len(point_set.points), n, replace=False)
    return indices
def four_point_congruent_sets(shape1, shape2, delta, color_threshold=50, iterations=100000):
    transformations = []
    congruent_count = 0  # For debugging
    
    points1 = np.asarray(shape1.points)
    points2 = np.asarray(shape2.points)
    colors1 = np.asarray(shape1.colors)
    colors2 = np.asarray(shape2.colors)
    
    for _ in range(iterations):
        base1_indices = random_base(shape1)
        base2_indices = random_base(shape2)

        if is_congruent(base1_indices, base2_indices, points1, points2, colors1, colors2, delta, color_threshold):
            congruent_count += 1  # For debugging
            source = o3d.geometry.PointCloud()
            source.points = o3d.utility.Vector3dVector(points1[list(base1_indices)])
            source.colors = o3d.utility.Vector3dVector(colors1[list(base1_indices)])
            
            target = o3d.geometry.PointCloud()
            target.points = o3d.utility.Vector3dVector(points2[list(base2_indices)])
            target.colors = o3d.utility.Vector3dVector(colors2[list(base2_indices)])
            
            transformation = estimate_transformation(source, target, delta)
            transformations.append(transformation)
    
    print(f"Total congruent pairs found: {congruent_count}")  # For debugging

    # Here we're simply returning the first transformation found for simplicity, but you could employ a voting scheme or other validation step
    return transformations[0] if transformations else None

# Load point clouds
# pcd1 = o3d.io.read_point_cloud("ply_test/frag_115.ply")
# pcd2 = o3d.io.read_point_cloud("ply_test/frag_116.ply")
pcd1 = copy.deepcopy(source_point_cloud)
pcd2 = copy.deepcopy(target_point_cloud)
pcd1_t = copy.deepcopy(pcd1)

alignment = four_point_congruent_sets(pcd1, pcd2, 0.5)
print(alignment)
# o3d.visualization.draw_geometries([pcd1, pcd2],width=800, height=600)

if alignment is not None:
    pcd1_t.transform(alignment)
    o3d.visualization.draw_geometries([pcd1_t, pcd2], width=800, height=600)
else:
    print("No alignment found.")

Total congruent pairs found: 100000
[[-0.20173081 -0.95145942 -0.23244281  0.13826579]
 [ 0.97726206 -0.17971022 -0.11253043  0.16486165]
 [ 0.06529579 -0.2498584   0.96607828  0.08125768]
 [ 0.          0.          0.          1.        ]]


**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='DL'></a>
# 7. Deep Learning Approaches
[https://pytorch.org/get-started/locally/]

Traditional point cloud registration methods, like Iterative Closest Point (ICP), work well when the initial alignment between the two point clouds is close to the correct solution. However, deep learning methods can be more robust and work well even without a good initial alignment.

Here are a few popular deep learning methods for point cloud registration:

A. **PointNetLK**: This method is inspired by the classical Lucas & Kanade algorithm. It uses the PointNet architecture to extract features from point clouds, and these features are then used to compute a transformation.

B. **Deep Closest Point (DCP)**: DCP utilizes PointNet to extract features and then estimates the transformation between two point clouds using a Sinkhorn layer.

C. **PCRNet**: This is another network inspired by PointNet that uses a simple architecture to predict the 6DoF transformation between two point clouds.


If you want to register the point clouds using one of these methods or another deep learning method, here's a general outline of what you would do:

- **Preprocess the Data**: Normalize and possibly downsample the point clouds.
- **Feature Extraction**: Use a deep network (like PointNet) to extract features from the point clouds.
- **Transformation Estimation**: Use the extracted features to estimate the transformation (rotation and translation) between the source and target point clouds. This might involve using a Sinkhorn layer (like in DCP) or another technique.
- **Refinement (Optional)**: Use a traditional method, like ICP, to refine the alignment.
- **Apply Transformation**: Use the estimated transformation to align the source to the target.



[(BACK TO TOP)](#TOP)

In [5]:
import torch
num_gpus = torch.cuda.device_count()
print("torch.cuda.is_available:",torch.cuda.is_available())
print(f"Number of GPUs available: {num_gpus}")
print("version:",torch.version.cuda)

torch.cuda.is_available: True
Number of GPUs available: 1
version: 11.7


In [6]:
%run Mylib_import.ipynb
pcd1 = copy.deepcopy(source_point_cloud)
pcd2 = copy.deepcopy(target_point_cloud)
# pcd1 = o3d.io.read_point_cloud("ply_test/frag_115.ply")
# pcd2 = o3d.io.read_point_cloud("ply_test/frag_116.ply")
# pcd1 = o3d.io.read_point_cloud("ply_test/00520-depth.ply")#
# pcd2 = o3d.io.read_point_cloud("ply_test/00540-depth.ply")#
# pcd1 = o3d.io.read_point_cloud("ply_test/bun000.ply")##
# pcd2 = o3d.io.read_point_cloud("ply_test/bun045.ply")##
######ModelNet40#######
# mesh1 = o3d.io.read_triangle_mesh("ply_test/ModelNet40/airplane/test/airplane_0627.off")
# mesh2 = o3d.io.read_triangle_mesh("ply_test/ModelNet40/airplane/train/airplane_0002.off")
# pcd1 = mesh1.sample_points_poisson_disk(number_of_points=1000)  
# pcd2 = mesh2.sample_points_poisson_disk(number_of_points=1000)
# ############################
# # rand_init = random_rotation_matrix_around_centroid(pcd1)
# # pcd1.transform(rand_init)
# pcd1.paint_uniform_color([1, 0, 0])##
# pcd2.paint_uniform_color([0, 0, 1])##
pcd1_t = copy.deepcopy(pcd1)
o3d.visualization.draw_geometries([pcd1, pcd2], width=800, height=600)

<a id='PointLK'></a>
**A. PointNetLK** 

https://github.com/hmgoforth/PointNetLK (conda activate pointnetlk_env)

PointNetLK is a point cloud registration method that brings together ideas from the classical Lucas & Kanade (LK) algorithm and the deep learning PointNet architecture. 

[(BACK TO TOP)](#TOP)

**with pre-trained models**
!!! transformation is failed, please consider Spatial Transformer Network (STN) following the paper!!!

In [6]:
import open3d as o3d
import torch
import numpy as np
import copy
from PointNetLK import ptlk

PointLK = ptlk.pointlk.PointLK
ptnet = ptlk.pointnet.PointNet_features()

# Load the state_dict from the file
ptnet.load_state_dict(torch.load("PointNetLK/output_pth/18092023/pointnet_200ite_feat_best.pth", map_location='cpu'))
# ptnet.load_state_dict(torch.load("C:/Users/msari/Research/Experiment/PointNetLK_vinits/pretrained/best_ptnet_model.t7", map_location='cpu'))

# Load the pretrained model
model = PointLK(ptnet=ptnet)  
model.load_state_dict(torch.load("PointNetLK/output_pth/18092023/pointlk_200ite_model_best.pth"), strict=False)
# model.load_state_dict(torch.load("C:/Users/msari/Research/Experiment/PointNetLK_vinits/pretrained/best_model.t7"), strict=False)
# model.load_state_dict(torch.load("C:/Users/msari/Research/Experiment/PointNetLK_vinits/checkpoints/exp_01/models/best_model.t7"), strict=False)

model.eval()


# Load point clouds 
source = copy.deepcopy(pcd1)
target = copy.deepcopy(pcd2)
pcd1_t = copy.deepcopy(pcd1)

# Convert the point clouds to PyTorch tensors
# Assuming your point clouds are of shape [N, 3]
source_tensor = torch.tensor(source.points, dtype=torch.float32).unsqueeze(0)  # Shape [1, N, 3]
target_tensor = torch.tensor(target.points, dtype=torch.float32).unsqueeze(0)  # Shape [1, N, 3]

# Pass the source and target tensors through the model
with torch.no_grad():
    model(target_tensor, source_tensor)

# Extract the transformation matrix
transformation_matrix = model.g.squeeze().cpu().numpy()

print("Transformation matrix:")
print(transformation_matrix)

pcd1_t.transform(transformation_matrix)
o3d.visualization.draw_geometries([source, target], width=800, height=600)
o3d.visualization.draw_geometries([pcd1_t, target], width=800, height=600)

C:\Users\msari\.conda\envs\pointnetlk_env\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)


Transformation matrix:
[[ 0.8728314   0.0239351  -0.4874344   0.03725404]
 [-0.02430457  0.9996891   0.00556772 -0.00210274]
 [ 0.48741615  0.00698719  0.87314177  0.0351776 ]
 [ 0.          0.          0.          1.        ]]


In [7]:
o3d.visualization.draw_geometries([pcd1_t, target], width=800, height=600)

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<a id='GA-based'></a>
# **8. GA-Based Optimization for Color Point Cloud Registration**

[(BACK TO TOP)](#TOP)

**1. GA-based on geometric positions/ GA-based on colors information**

In [57]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation
from sklearn.neighbors import NearestNeighbors
import colorsys

# Initialization
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        # Random initialization of rotation matrix and translation vector
        # Pass an appropriate value for rotation_range when calling random_rotation_matrix
        rotation_matrix = random_rotation_matrix(np.pi)  # Example: Full range of rotation
        translation_vector = random_translation_vector()
        
        # Combine rotation matrix and translation vector into a 4x4 transformation matrix
        transformation_matrix = np.eye(4)
        transformation_matrix[:3, :3] = rotation_matrix
        transformation_matrix[:3, 3] = translation_vector
        
        # Append the transformation matrix to the population
        population.append(transformation_matrix)
    return population

def random_rotation_matrix():
    # Generating random Euler angles (in radians)
    alpha = np.random.uniform(0, 2 * np.pi)
    beta = np.random.uniform(0, 2 * np.pi)
    gamma = np.random.uniform(0, 2 * np.pi)
    
    # Constructing the rotation matrix using the Euler angles
    R_alpha = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                        [np.sin(alpha), np.cos(alpha), 0],
                        [0, 0, 1]])
    R_beta = np.array([[1, 0, 0],
                       [0, np.cos(beta), -np.sin(beta)],
                       [0, np.sin(beta), np.cos(beta)]])
    R_gamma = np.array([[np.cos(gamma), 0, np.sin(gamma)],
                        [0, 1, 0],
                        [-np.sin(gamma), 0, np.cos(gamma)]])
    
    # Combining the rotation matrices
    rotation_matrix = np.dot(R_alpha, np.dot(R_beta, R_gamma))
    return rotation_matrix

def random_translation_vector():
    # Define the range of possible translations
    translation_range = (-1, 1)  # Example range; adjust based on your requirements
    
    # Generate random translation for each axis within the defined range
    tx = np.random.uniform(translation_range[0], translation_range[1])
    ty = np.random.uniform(translation_range[0], translation_range[1])
    tz = np.random.uniform(translation_range[0], translation_range[1])
    
    # Combine translations into a vector
    translation_vector = np.array([tx, ty, tz])
    return translation_vector
#__________________
# Fitness function
##for geometric
def evaluate_fitness_rmse(individual):
    # Apply transformation to source point cloud
    transformed_source = source.transform(individual)
    
    # Evaluate alignment error
    # One way is to use the evaluate_registration function provided by Open3D
    # which computes the RMSE of point-to-point distances after registration.
    evaluation = o3d.registration.evaluate_registration(
        transformed_source, target, max_correspondence_distance=0.1)  # adjust the max_correspondence_distance
    
    # Here, we use 1 / (1 + error) as the fitness function.
    # You might want to use a different function that better suits your needs.
    fitness = 1 / (1 + evaluation.inlier_rmse)
    return fitness

##for color
def evaluate_fitness_joint(individual, geom_weight=0.7):
     # Check if weights are valid
    if not 0 <= geom_weight <= 1:
        raise ValueError("geom_weight (lambda) must be between 0 and 1 inclusive")
    
    color_weight = 1 - geom_weight  # Calculate color_weight as (1 - lambda)
    
    # Apply transformation to source point cloud
    transformed_source = source.transform(individual)
    
    # Convert points to numpy array for nearest neighbors search
    source_points = np.asarray(transformed_source.points)
    target_points = np.asarray(target.points)
    
    # Find nearest neighbors in target for each point in transformed source
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(target_points)
    distances, indices = nbrs.kneighbors(source_points)
    
    # Compute geometric error (adjust this as needed)
    geom_error = np.mean(distances)
    
    # Compute color error for corresponding points
    source_colors = np.asarray(transformed_source.colors)
    target_colors = np.asarray(target.colors)[indices.flatten()]
    color_difference = np.linalg.norm(source_colors - target_colors, axis=1)
    average_color_difference = np.mean(color_difference)
    
    # Combine geometric and color error into a single fitness value with adjustable weights
    fitness = geom_weight * (1 / (1 + geom_error)) + color_weight * (1 / (1 + average_color_difference))
    return fitness
#__________________
# Selection
def select_individuals(population, selection = 'Roulette'):
    if selection == 'Roulette': # Roulette Wheel Selection
        print("Roulette Wheel Selection")
        # Calculate fitness values for each individual in the population
        fitness_values = np.array([evaluate_fitness_rmse(ind) for ind in population])

        # Normalize fitness values so they sum to 1
        fitness_values /= fitness_values.sum()

        # Select individuals with probability proportional to their normalized fitness
        selected_indices = np.random.choice(len(population), size=len(population), p=fitness_values)
        selected_individuals = [population[idx] for idx in selected_indices]
        
    elif selection == 'Tournament': # Tournament Selection
        print("Tournament Selection")
        tournament_size=3
        selected_individuals = []
        pop_size = len(population)

        for _ in range(pop_size):
            # Randomly select tournament_size indices from the population
            tournament_indices = np.random.choice(pop_size, size=tournament_size, replace=False)

            # Get the individuals corresponding to these indices
            tournament_contestants = [population[idx] for idx in tournament_indices]

            # Select the individual with the highest fitness from the tournament
            best_individual = max(tournament_contestants, key=evaluate_fitness_rmse)
            selected_individuals.append(best_individual)     
            
    return selected_individuals
#__________________
# Crossover
def crossover(parent1, parent2):
    # Extract rotation matrices and translation vectors from parents
    R1 = parent1[:3, :3]
    t1 = parent1[:3, 3]
    R2 = parent2[:3, :3]
    t2 = parent2[:3, 3]
    
    # Convert rotation matrices to quaternions
    quat1 = Rotation.from_matrix(R1).as_quat()
    quat2 = Rotation.from_matrix(R2).as_quat()
    
    # Use the slerp function to interpolate between quat1 and quat2
    slerp_quat = slerp(quat1, quat2, t=0.5)
    
    # Convert the result back to a rotation matrix
    child_R = Rotation.from_quat(slerp_quat).as_matrix()
    
    # Linear interpolation between the two translation vectors
    child_t = (t1 + t2) / 2
    
    # Combine child rotation matrix and translation vector into a transformation matrix
    child1 = np.eye(4)
    child1[:3, :3] = child_R
    child1[:3, 3] = child_t
    
    # For demonstration purposes, using the same child as child2. 
    # You might want to generate a different second child based on your crossover logic.
    child2 = np.copy(child1)  
    
    return child1, child2

def slerp(quat1, quat2, t):
    """
    Spherical linear interpolation between two quaternions.
    """
    # Compute the cosine of the angle between the two quaternions
    dot_product = np.dot(quat1, quat2)
    
    # If the dot product is negative, slerp won't take the shorter path
    if dot_product < 0.0:
        quat2 = -quat2
        dot_product = -dot_product
    
    # For dot product approximately 1, the two quaternions are very close
    # to avoid division by zero, use linear interpolation
    if dot_product > 0.9995:
        result = quat1 + t * (quat2 - quat1)
        return result / np.linalg.norm(result)
    
    # Compute the quaternion of the spherical linear interpolation
    angle = np.arccos(dot_product)
    return (np.sin((1.0 - t) * angle) * quat1 + np.sin(t * angle) * quat2) / np.sin(angle)

#__________________
# Mutation
def mutate(individual, translation_mutation_range=0.1, rotation_mutation_range=np.pi/36):  
    # Extract rotation matrix and translation vector from the individual
    R = individual[:3, :3]
    t = individual[:3, 3]
    
    # Apply small random translations
    t += np.random.uniform(-translation_mutation_range, translation_mutation_range, 3)
    
    # Apply small random rotations (you might use a similar approach as in random_rotation_matrix function)
    small_rotation = random_rotation_matrix(rotation_mutation_range)
    mutated_R = np.dot(small_rotation, R)
    
    # Combine mutated rotation matrix and translation vector into a transformation matrix
    mutated_individual = np.eye(4)
    mutated_individual[:3, :3] = mutated_R
    mutated_individual[:3, 3] = t
    
    return mutated_individual

def random_rotation_matrix(rotation_range):
    # Generate a random axis by normalizing a random vector
    axis = np.random.rand(3)
    axis /= np.linalg.norm(axis)
    
    # Generate a random angle within the specified range
    angle = np.random.uniform(-rotation_range, rotation_range)
    
    # Compute the rotation matrix using the axis-angle representation
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    rotation_matrix = np.array([
        [cos_angle + axis[0]**2 * (1 - cos_angle), axis[0] * axis[1] * (1 - cos_angle) - axis[2] * sin_angle, axis[0] * axis[2] * (1 - cos_angle) + axis[1] * sin_angle],
        [axis[1] * axis[0] * (1 - cos_angle) + axis[2] * sin_angle, cos_angle + axis[1]**2 * (1 - cos_angle), axis[1] * axis[2] * (1 - cos_angle) - axis[0] * sin_angle],
        [axis[2] * axis[0] * (1 - cos_angle) - axis[1] * sin_angle, axis[2] * axis[1] * (1 - cos_angle) + axis[0] * sin_angle, cos_angle + axis[2]**2 * (1 - cos_angle)]
    ])
    
    return rotation_matrix

In [60]:
# Main code for Geo or Color + GA
source = copy.deepcopy(source_point_cloud)
target = copy.deepcopy(target_point_cloud)

# GA parameters
pop_size = 50  # population size
num_gen = 100  # number of generations
mutation_rate = 0.1  # probability of mutation for each individual

# Main GA loop
population = initialize_population(pop_size)
for gen in range(num_gen):
    print(f"Generation {gen + 1}/{num_gen}")
    
    # Evaluate fitness of individuals
    # Select the appropriate fitness evaluation function
    # You need to define the source and target point clouds before evaluating fitness
#     fitness_values = [evaluate_fitness_rmse(ind) for ind in population]
    # OR
    fitness_values = [evaluate_fitness_joint(ind, geom_weight=0.7) for ind in population]
    
    # Select individuals for reproduction 'Roulette' or 'Tournament'
    selected_individuals = select_individuals(population, selection = 'Tournament')
    
    # Create next generation through crossover and mutation
    new_population = []
    for i in range(0, pop_size, 2):
        parent1 = selected_individuals[i]
        parent2 = selected_individuals[i + 1]
        child1, child2 = crossover(parent1, parent2)
        
        # Apply mutation with a given probability
        if np.random.rand() < mutation_rate:
            child1 = mutate(child1)
        if np.random.rand() < mutation_rate:
            child2 = mutate(child2)
        
        new_population.append(child1)
        new_population.append(child2)
    
    # Replace old population with new population
    population = new_population

# Select the best individual from the final population
# best_individual = max(population, key=evaluate_fitness_rmse)
# OR use the appropriate fitness evaluation function
best_individual = max(population, key=evaluate_fitness_joint)

print("Best Transformation Matrix:")
print(best_individual)

# Visualization (ensure you have Open3D installed and configured)
t_source = copy.deepcopy(source_point_cloud)
o3d.visualization.draw_geometries([source_point_cloud, target_point_cloud], width=800, height=600)
t_source.transform(best_individual)
o3d.visualization.draw_geometries([t_source, target_point_cloud], width=800, height=600)

Generation 1/100
Tournament Selection
Generation 2/100
Tournament Selection
Generation 3/100
Tournament Selection
Generation 4/100
Tournament Selection
Generation 5/100
Tournament Selection
Generation 6/100
Tournament Selection
Generation 7/100
Tournament Selection
Generation 8/100
Tournament Selection
Generation 9/100
Tournament Selection
Generation 10/100
Tournament Selection
Generation 11/100
Tournament Selection
Generation 12/100
Tournament Selection
Generation 13/100
Tournament Selection
Generation 14/100
Tournament Selection
Generation 15/100
Tournament Selection
Generation 16/100
Tournament Selection
Generation 17/100
Tournament Selection
Generation 18/100
Tournament Selection
Generation 19/100
Tournament Selection
Generation 20/100
Tournament Selection
Generation 21/100
Tournament Selection
Generation 22/100
Tournament Selection
Generation 23/100
Tournament Selection
Generation 24/100
Tournament Selection
Generation 25/100
Tournament Selection
Generation 26/100
Tournament Selec

**-----------------------------------------------------------**

**2. following papers: Genetic Algorithm-Based Optimization for Color Point Cloud Registration**

In [46]:
def rgb_to_hsv(rgb):
    """Convert RGB color to HSV."""
    return np.array([colorsys.rgb_to_hsv(c[0], c[1], c[2]) for c in rgb])

def global_brightness_transformation(source_hsv, target_hsv):
    # Apply logarithmic brightness transformation to source and target
    transformed_source_hsv = source_hsv.copy()
    transformed_target_hsv = target_hsv.copy()
    
    # Apply logarithmic brightness transformation to the V channel of source
    transformed_source_hsv[:, 2] = np.log(transformed_source_hsv[:, 2] + 1)
    
    # Find max and min of transformed target V channel
    max_target_v = np.max(np.log(transformed_target_hsv[:, 2] + 1))
    min_target_v = np.min(np.log(transformed_target_hsv[:, 2] + 1))
    
    # Normalize the transformed source V channel to the [min_target_v, max_target_v] range
    max_source_v = np.max(transformed_source_hsv[:, 2])
    min_source_v = np.min(transformed_source_hsv[:, 2])
    transformed_source_hsv[:, 2] = (transformed_source_hsv[:, 2] - min_source_v) / (max_source_v - min_source_v) * (max_target_v - min_target_v) + min_target_v
    
    return transformed_source_hsv, target_hsv

def saturation_feedback(source_hsv, target_hsv):
    """
    Adjust the saturation of source HSV colors based on target HSV colors.
    This is a simplistic approach; you might need a more sophisticated mechanism.
    """
    # Calculate the average saturation of source and target
    avg_saturation_source = np.mean(source_hsv[:, 1])
    avg_saturation_target = np.mean(target_hsv[:, 1])
    
    # Calculate the saturation adjustment factor
    saturation_factor = avg_saturation_target / avg_saturation_source

    # Apply the saturation adjustment to the source colors
    adjusted_source_hsv = source_hsv.copy()
    adjusted_source_hsv[:, 1] = np.clip(adjusted_source_hsv[:, 1] * saturation_factor, 0, 1)
    
    return adjusted_source_hsv, target_hsv

def hsv_continuation(hsv_colors):
    """Apply HSV continuation to a list of HSV colors."""
    # Assuming hsv_colors is a numpy array
    # You might need a more sophisticated approach depending on your specific needs
    for i in range(1, len(hsv_colors)):
        while hsv_colors[i, 0] - hsv_colors[i-1, 0] > 0.5:
            hsv_colors[i, 0] -= 1
        while hsv_colors[i, 0] - hsv_colors[i-1, 0] < -0.5:
            hsv_colors[i, 0] += 1
    return hsv_colors

#__________________
# Fitness function
def evaluate_fitness_HSV(individual, geom_weight=0.7):  
#     print(f"geom_weight:{geom_weight}")
    # Check if weights are valid
    if not 0 <= geom_weight <= 1:
        raise ValueError("geom_weight (lambda) must be between 0 and 1 inclusive")
    
    color_weight = 1 - geom_weight  # Calculate color_weight as (1 - lambda)
    
    # Apply transformation to source point cloud
    transformed_source = source.transform(individual)
    
     # Convert points to numpy array for nearest neighbors search
    source_points = np.asarray(transformed_source.points)
    target_points = np.asarray(target.points)
    
    # Find nearest neighbors in target for each point in transformed source
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(target_points)
    distances, indices = nbrs.kneighbors(source_points)
    
    # Compute geometric error (adjust this as needed)
    geom_error = np.mean(distances)
    
    # Compute color error for corresponding points in continuous HSV space
    source_colors = rgb_to_hsv(np.asarray(transformed_source.colors))
    target_colors = rgb_to_hsv(np.asarray(target.colors)[indices.flatten()])
    
    # Apply global brightness transformation
    source_colors, target_colors = global_brightness_transformation(source_colors, target_colors)
    
    # Apply saturation feedback
    source_colors, target_colors = saturation_feedback(source_colors, target_colors)
    
    # Apply HSV continuation
    source_colors = hsv_continuation(source_colors)
    
    color_difference = np.linalg.norm(source_colors - target_colors, axis=1)
    average_color_difference = np.mean(color_difference)
    
    # Combine geometric and color error into a single fitness value with adjustable weights
    fitness = geom_weight * (1 / (1 + geom_error)) + color_weight * (1 / (1 + average_color_difference))
    
    return fitness


In [56]:
# Main code for HSV+GA
source = copy.deepcopy(source_point_cloud)
target = copy.deepcopy(target_point_cloud)

# GA parameters
pop_size = 50  # population size
num_gen = 100  # number of generations
mutation_rate = 0.1  # probability of mutation for each individual

# Main GA loop
population = initialize_population(pop_size)
for gen in range(num_gen):
    print(f"Generation {gen + 1}/{num_gen}")
    
    # Evaluate fitness of individuals
    fitness_values = [evaluate_fitness_HSV(ind, geom_weight=0.7) for ind in population]
    
    # Select individuals for reproduction 'Roulette' or 'Tournament'
    selected_individuals = select_individuals(population, selection='Tournament')
    
    # Create next generation through crossover and mutation
    new_population = []
    for i in range(0, pop_size, 2):
        parent1 = selected_individuals[i]
        parent2 = selected_individuals[i + 1]
        child1, child2 = crossover(parent1, parent2)
        
        # Apply mutation with a given probability
        if np.random.rand() < mutation_rate:
            child1 = mutate(child1)
        if np.random.rand() < mutation_rate:
            child2 = mutate(child2)
        
        new_population.append(child1)
        new_population.append(child2)
    
    # Replace old population with new population
    population = new_population

# Select the best individual from the final population
best_individual_HSV = max(population, key=evaluate_fitness_HSV)

print("Best Transformation Matrix:", best_individual_HSV)

# Visualization (ensure you have Open3D installed and configured)
t_source = copy.deepcopy(source_point_cloud)
t_source.transform(best_individual_HSV)
o3d.visualization.draw_geometries([t_source, target_point_cloud], width=800, height=600)

Generation 1/100


C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regi

Tournament Selection
Generation 2/100


C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regi

Tournament Selection
Generation 3/100


...

Tournament Selection
Generation 100/100


C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regi

Tournament Selection


C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regist\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\msari\.conda\envs\pc_regi

Best Transformation Matrix: [[ 0.29361854  0.88773296 -0.35456784 -0.22337475]
 [-0.0208987   0.3767895   0.92606313  0.05627276]
 [ 0.9556942  -0.26449929  0.12918485 -0.13406523]
 [ 0.          0.          0.          1.        ]]


# **--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-----------------------**

**-----------------------**